__Исследование взаимосвязи погоды и ДТП в городе Москва.__
В данном исследовании рассматривается вопрос взаимосвязи ДТП и погодных характеристик для города Москвы за 2015-2021 года. В качестве исходных данных для ДТП взяты открытые данные сайтов dtp-stat.ru, для погоды - открытые данные сайта rp5.ru для станции метеонаблюдения Балчуг (центр Москвы).

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
#import seaborn as sns
#import matplotlib.pyplot as plt
#import numpy as np
#import json
#from io import BytesIO
#import requests
#Изменение ограничений на длину вывода информации
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 50)
#import warnings
#warnings.filterwarnings('ignore') #Много предупреждений о копировании датасетов через .loc, отключил
import geopandas as gpd

расшифровка столбцов:
- 'Местное время в Москве (центр, Балчуг)',
-  'T' - 'Температура воздуха (градусы Цельсия) на высоте 2 метра над поверхностью земли'
- 'Po' - Атмосферное давление на уровне станции (миллиметры ртутного столба)- 
-  'P' - Атмосферное давление, приведенное к среднему уровню моря (миллиметры ртутного столба)
-  'Pa' - Барическая тенденция: изменение атмосферного давления за последние три часа (миллиметры ртутного столба)'
-  'U' - Относительная влажность (%) на высоте 2 метра над поверхностью земли
-  'DD'- Направление ветра (румбы) на высоте 10-12 метров над земной поверхностью, осредненное за 10-минутный период, непосредственно предшествовавший сроку наблюдения'
-  'Ff' - Cкорость ветра на высоте 10-12 метров над земной поверхностью, осредненная за 10-минутный период, непосредственно предшествовавший сроку наблюдения (метры в секунду)
-  'ff10'- 'Максимальное значение порыва ветра на высоте 10-12 метров над земной поверхностью за 10-минутный период, непосредственно предшествующий сроку наблюдения (метры в секунду)
-  'ff3'- Максимальное значение порыва ветра на высоте 10-12 метров над земной поверхностью за период между сроками (метры в секунду)
-  'N'- Общая облачность
-  'WW' - Текущая погода, сообщаемая с метеорологической станции
-  'W1'- Прошедшая погода между сроками наблюдения 1
-  'W2'- Прошедшая погода между сроками наблюдения 2
-  'Tn'- Минимальная температура воздуха (градусы Цельсия) за прошедший период (не более 12 часов)
-  'Tx' - Максимальная температура воздуха (градусы Цельсия) за прошедший период (не более 12 часов)
-  'Cl' - Слоисто-кучевые, слоистые, кучевые и кучево-дождевые облака
-  'Nh' - Количество всех наблюдающихся облаков Cl или, при отсутствии облаков Cl, количество всех наблюдающихся облаков Cm
-  'H' - Высота основания самых низких облаков (м)
-  'Cm'- Высококучевые, высокослоистые и слоисто-дождевые облака
-  'Ch' - Перистые, перисто-кучевые и перисто-слоистые облака
-  'VV'- Горизонтальная дальность видимости (км)
-  'Td' - Температура точки росы на высоте 2 метра над поверхностью земли (градусы Цельсия)
-  'RRR' - Количество выпавших осадков (миллиметры)
-  'tR' -Период времени, за который накоплено указанное количество осадков (часы)
-  'E'- Состояние поверхности почвы без снега или измеримого ледяного покрова,
-  'Tg' -Минимальная температура поверхности почвы за ночь. (градусы Цельсия),
-  "E'" -Состояние поверхности почвы со снегом или измеримым ледяным покровом,
-  'sss' - Высота снежного покрова (см)]

Сложность 1: определить, какие из множества значений действительно влияют на целевой параметр

In [2]:
weather = pd.read_csv('BALCHUG.01.01.2015.04.01.2022.1.0.0.ru.utf8.00000000.csv', skiprows=6, sep=';', index_col=False)
weather.rename(columns={'Местное время в Москве (центр, Балчуг)':'время'}, inplace=True)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17142 entries, 0 to 17141
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   время   17142 non-null  object 
 1   T       17140 non-null  float64
 2   Po      17142 non-null  float64
 3   P       17142 non-null  float64
 4   Pa      2463 non-null   float64
 5   U       17140 non-null  float64
 6   DD      17140 non-null  object 
 7   Ff      17141 non-null  float64
 8   ff10    71 non-null     float64
 9   ff3     562 non-null    float64
 10  N       17128 non-null  object 
 11  WW      17142 non-null  object 
 12  W1      4960 non-null   object 
 13  W2      4960 non-null   object 
 14  Tn      4181 non-null   float64
 15  Tx      2149 non-null   float64
 16  Cl      14626 non-null  object 
 17  Nh      14626 non-null  object 
 18  H       12659 non-null  object 
 19  Cm      9474 non-null   object 
 20  Ch      6356 non-null   object 
 21  VV      14893 non-null  float64
 22

In [3]:
weather.head(2)

,время,T,Po,P,Pa,U,DD,Ff,ff10,ff3,N,WW,W1,W2,Tn,Tx,Cl,Nh,H,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
0,04.01.2022 15:00,-6.3,741.1,752.9,NaN,77.0,"Штиль, безветрие",0.0,NaN,NaN,100%.,,NaN,NaN,NaN,NaN,"Слоисто-кучевые, образовавшиеся не из кучевых.",50%.,1000-1500,"Высококучевые просвечивающие, полосами, либо о...",NaN,10.0,-9.7,NaN,NaN,NaN,NaN,NaN,NaN
1,04.01.2022 12:00,-7.0,740.8,752.7,NaN,83.0,"Ветер, дующий с востоко-юго-востока",1.0,NaN,NaN,100%.,,NaN,NaN,NaN,NaN,"Слоисто-кучевые, образовавшиеся не из кучевых.",100%.,600-1000,NaN,NaN,10.0,-9.5,NaN,NaN,NaN,NaN,NaN,NaN


Изучим json массив. Для начала прочитаем его в датафрейм data

In [4]:
data = pd.read_json('moskva.geojson')
data.head()


,type,features
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
2,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
3,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
4,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."


In [5]:
data = pd.json_normalize(data['features'])

In [6]:
data.head()

,type,geometry.type,geometry.coordinates,properties.id,properties.tags,properties.light,properties.point.lat,properties.point.long,properties.nearby,properties.region,properties.scheme,properties.address,properties.weather,properties.category,properties.datetime,properties.severity,properties.vehicles,properties.dead_count,properties.participants,properties.injured_count,properties.parent_region,properties.road_conditions,properties.participants_count,properties.participant_categories
0,Feature,Point,"[37.770245, 55.667499]",2575117,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено",55.667499,37.770245,[Автостоянка (не отделённая от проезжей части)...,Люблино,910,"г Москва, ул Верхние Поля, 39",[Ясно],Наезд на стоящее ТС,2021-05-21 00:35:00,Легкий,"[{'year': 1993, 'brand': 'TOYOTA', 'color': 'С...",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'vio...",2,Москва,[Сухое],3,"[Пешеходы, Все участники]"
1,Feature,Point,"[37.553995, 55.669411]",2575131,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено",55.669411,37.553995,[Крупный торговый объект (являющийся объектом ...,Черемушки,960,"г Москва, ул Профсоюзная, 56",[Ясно],Падение пассажира,2021-05-14 22:30:00,Легкий,"[{'year': 2018, 'brand': 'FORD', 'color': 'Мно...",0,[],1,Москва,[Сухое],2,[Все участники]
2,Feature,Point,"[37.577877, 55.752538]",2575134,"[Дорожно-транспортные происшествия, ДТП и пост...",Светлое время суток,55.752538,37.577877,"[Многоквартирные жилые дома, Зоны отдыха, Адми...",Арбат,820,"г Москва, ул Новый Арбат, 27","[Пасмурно, Дождь]",Наезд на пешехода,2021-02-10 14:40:00,Легкий,"[{'year': 2019, 'brand': 'MERCEDES', 'color': ...",0,"[{'role': 'Пешеход', 'gender': 'Мужской', 'vio...",1,Москва,[Мокрое],2,"[Дети, Пешеходы, Все участники]"
3,Feature,Point,"[37.322574, 55.802463]",2575136,[Дорожно-транспортные происшествия],Светлое время суток,55.802463,37.322574,"[Многоквартирные жилые дома, АЗС, Нерегулируем...",Спецтрассы,070,"г Москва, А-109 А-109 Ильинское шоссе, 3 км",[Пасмурно],Столкновение,2021-04-29 07:45:00,Легкий,"[{'year': 2018, 'brand': 'ГАЗ', 'color': 'Белы...",0,[],1,Москва,[Сухое],4,[Все участники]
4,Feature,Point,"[37.347625, 55.635167]",2599457,[Дорожно-транспортные происшествия],Светлое время суток,55.635167,37.347625,"[Многоквартирные жилые дома, АЗС, Остановка об...",Ново-Переделкино,None,"г Москва, ш Боровское, 52",[Ясно],Опрокидывание,2021-07-18 15:09:00,Легкий,[],0,"[{'role': 'Водитель', 'gender': None, 'violati...",1,Москва,[Сухое],1,[Все участники]


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62568 entries, 0 to 62567
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   type                               62568 non-null  object 
 1   geometry.type                      62568 non-null  object 
 2   geometry.coordinates               62568 non-null  object 
 3   properties.id                      62568 non-null  int64  
 4   properties.tags                    62568 non-null  object 
 5   properties.light                   62568 non-null  object 
 6   properties.point.lat               62485 non-null  float64
 7   properties.point.long              62485 non-null  float64
 8   properties.nearby                  62568 non-null  object 
 9   properties.region                  62568 non-null  object 
 10  properties.scheme                  59137 non-null  object 
 11  properties.address                 58761 non-null  obj

Можем удалить излишние столбцы: 'type', 'geometry.type', 'properties.point.lat','properties.point.long'

In [8]:
data.drop(axis='columns', labels={'type', 'geometry.type', 'properties.point.lat','properties.point.long'}, inplace=True)

In [9]:
dtp_vehicles = pd.json_normalize(data.loc[:,'properties.vehicles'])
dtp_vehicles['properties.id'] = data.loc[:,'properties.id']


In [10]:
dtp_vehicles['car_count'] = dtp_vehicles.loc[:].count(axis=1)-2 # добавим столбец с количеством участвующих авто
dtp_vehicles.head()
dtp_vehicles[0][0]['participants']

[{'role': 'Водитель',
  'gender': 'Мужской',
  'violations': ['Несоответствие скорости конкретным условиям движения',
   'Отказ водителя от прохождения медицинского освидетельствования на состояние опьянения'],
  'health_status': 'Не пострадал',
  'years_of_driving_experience': 2}]

In [11]:
dtp_part = pd.json_normalize(data.loc[:,'properties.participants'])
dtp_part['properties.id'] = data.loc[:,'properties.id'] #Добавим ключ 
dtp_part['part_count'] = dtp_part.loc[:].count(axis=1)-1 # Добавим столбец с количеством участвующих в дтп людей
dtp_part.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,properties.id,part_count
0,"{'role': 'Пешеход', 'gender': 'Женский', 'viol...","{'role': 'Пешеход, перед ДТП находившийся в (н...",None,None,None,None,None,None,None,None,None,None,None,2575117,2
1,None,None,None,None,None,None,None,None,None,None,None,None,None,2575131,0
2,"{'role': 'Пешеход', 'gender': 'Мужской', 'viol...",None,None,None,None,None,None,None,None,None,None,None,None,2575134,1
3,None,None,None,None,None,None,None,None,None,None,None,None,None,2575136,0
4,"{'role': 'Водитель', 'gender': None, 'violatio...",None,None,None,None,None,None,None,None,None,None,None,None,2599457,1


## Преобразование категорий
Для корректной связки данных о погоде и данных о дтп необходимо преобразовать стобцы с информацией о дате и времени. И так как данные о погоде идут с 3-х часовым лагом (кстати, проверим это)

In [12]:
data['properties.datetime'] = pd.to_datetime(arg=data['properties.datetime'], format='%Y-%m-%d %H:%M:%S') #2021-05-21 00:35:00
weather['время'] = pd.to_datetime(weather['время'], format='%d.%m.%Y %H:%M')
weather.head(1)

,время,T,Po,P,Pa,U,DD,Ff,ff10,ff3,N,WW,W1,W2,Tn,Tx,Cl,Nh,H,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
0,2022-01-04 15:00:00,-6.3,741.1,752.9,NaN,77.0,"Штиль, безветрие",0.0,NaN,NaN,100%.,,NaN,NaN,NaN,NaN,"Слоисто-кучевые, образовавшиеся не из кучевых.",50%.,1000-1500,"Высококучевые просвечивающие, полосами, либо о...",NaN,10.0,-9.7,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
set(weather['время'].dt.hour) 

{0, 3, 6, 9, 12, 15, 18, 21}

Действительно, во всем датасете временной интервал погоды равен трем часам, от 0 до 21 часа. Следовательно, нам необходимо все временные отметки данных о ДТП округлить к ближайшему значению.

In [14]:
data['properties.datetime'] = data['properties.datetime'].dt.round(freq='3H')

In [15]:
weather.loc[weather.sort_values('время', ascending=True
                               )['время'
                                ].diff(
                                ).apply(lambda x: x/pd.Timedelta('1 hour'))==6
      ].head()#groupby(weather['время'
               #        ].dt.hour
               #).count()


,время,T,Po,P,Pa,U,DD,Ff,ff10,ff3,N,WW,W1,W2,Tn,Tx,Cl,Nh,H,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
34,2021-12-31 09:00:00,-5.3,747.5,759.4,NaN,87.0,"Ветер, дующий с юго-юго-востока",1.0,NaN,NaN,100%.,Дымка.,Облака покрывали более половины неба в течение...,Облака покрывали более половины неба в течение...,-6.7,NaN,"Слоисто-кучевые, образовавшиеся не из кучевых.",100%.,600-1000,NaN,NaN,4.0,-7.2,Осадков нет,12.0,NaN,NaN,Ровный слой сухого рассыпчатого снега покрывае...,25.0
90,2021-12-24 06:00:00,-10.4,744.4,756.6,NaN,78.0,"Ветер, дующий с юго-юго-запада",1.0,NaN,NaN,100%.,,NaN,NaN,-11.2,NaN,"Слоисто-кучевых, слоистых, кучевых или кучево-...",100%.,"2500 или более, или облаков нет.",Высокослоистые непросвечивающие или слоисто-до...,NaN,10.0,-13.6,Осадков нет,12.0,NaN,NaN,NaN,NaN
120,2021-12-20 09:00:00,-7.4,732.7,744.6,NaN,86.0,"Ветер, дующий с северо-запада",3.0,NaN,NaN,100%.,Снег непрерывный слабый в срок наблюдения.,Снег и/или другие виды твердых осадков,Облака покрывали более половины неба в течение...,-7.4,NaN,"Слоисто-кучевых, слоистых, кучевых или кучево-...",100%.,300-600,Высокослоистые непросвечивающие или слоисто-до...,NaN,4.0,-9.4,2.0,12.0,NaN,NaN,Ровный слой сухого рассыпчатого снега покрывае...,24.0
303,2021-11-27 09:00:00,3.2,745.1,756.6,NaN,79.0,"Ветер, дующий с юго-юго-востока",1.0,NaN,NaN,100%.,,NaN,NaN,2.9,NaN,"Слоисто-кучевые, образовавшиеся не из кучевых.",100%.,600-1000,NaN,NaN,10.0,-0.1,Осадков нет,12.0,NaN,NaN,NaN,NaN
534,2021-10-29 09:00:00,7.3,757.6,769.1,NaN,70.0,"Ветер, дующий с юго-запада",1.0,NaN,NaN,100%.,,NaN,NaN,6.9,NaN,"Слоисто-кучевые, образовавшиеся не из кучевых.",100%.,600-1000,NaN,NaN,10.0,2.2,Осадков нет,12.0,Поверхность почвы влажная.,5.0,NaN,NaN


Определим, есть ли пропуски в данных о погоде, какой они длительности и сколько их

In [17]:
weather.sort_values('время', ascending=True
      ).groupby(weather.sort_values('время', ascending=True
                       )['время'
                       ].diff()
               ).count(
       )['время'
        ].rename(lambda x: x/pd.Timedelta('1 hour')
        ).reset_index(name='Количество диапазонов')


,время,Количество диапазонов
0,3.0,16951
1,6.0,174
2,9.0,5
3,12.0,2
4,15.0,1
5,21.0,1
6,27.0,6
7,9291.0,1


Отсутствующие значения погоды возьмем из набора данных погоды с ВДНХ

In [18]:
weather_vdnh = pd.read_csv('VDNH.01.01.2015.04.01.2022.1.0.0.ru.utf8.00000000.csv', skiprows=6, sep=';', index_col=False)
weather_vdnh.rename(columns={'Местное время в Москве (ВДНХ)':'время'}, inplace=True)
weather_vdnh['время'] = pd.to_datetime(weather_vdnh['время'], format='%d.%m.%Y %H:%M')
weather_vdnh['время'] = weather_vdnh['время'].dt.round(freq='3H') #Округлим временные отметки для их единообразия
weather_vdnh.head(1)


,время,T,Po,P,Pa,U,DD,Ff,ff10,ff3,N,WW,W1,W2,Tn,Tx,Cl,Nh,H,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
0,2022-01-04 21:00:00,-6.2,739.2,753.4,0.2,80.0,"Ветер, дующий с юга",1,NaN,NaN,100%.,Состояние неба в общем не изменилось.,Снег и/или другие виды твердых осадков,Облака покрывали более половины неба в течение...,NaN,-6.2,"Слоисто-кучевые, образовавшиеся не из кучевых.",100%.,1000-1500,NaN,NaN,13.0,-9.1,0.1,12.0,NaN,NaN,NaN,NaN


Добавим в weather данные с ВДНХ по отсутствующим пунктам.

In [20]:
weather = weather.append(weather_vdnh.loc[~weather_vdnh['время'].isin(weather['время'])], ignore_index=True)

In [21]:
weather.sort_values('время', ascending=True
      ).groupby(weather.sort_values('время', ascending=True
                       )['время'
                       ].diff()
               ).count(
       )['время'
        ].rename(lambda x: x/pd.Timedelta('1 hour')
        ).reset_index(name='Количество диапазонов')


,время,Количество диапазонов
0,3.0,20474
1,6.0,5
2,9.0,1


Оставшиеся отсутствующие значения можем проигнорировать, на общее качество данных они не повлияют. Можно приступать к объединению датафреймов

In [22]:
dtp_join = data.merge(weather, how='left', left_on='properties.datetime', right_on='время')
#Проверим, сколько пустых значений получилось в данных
dtp_join.loc[dtp_join['properties.datetime']!=dtp_join['время'], 'properties.datetime'].count()

11

Всего 11 ДТП остались без данных о погоде. Удалим их из датафрейма

In [33]:
dtp_join = dtp_join.loc[~dtp_join['время'].isna()]
#Проверим
dtp_join.loc[dtp_join['properties.datetime']!=dtp_join['время'], 'properties.datetime'].count()

0